In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [2]:
def create_folds(data, n_splits, random_state=None):
    data["fold"] = -1
    data = data.sample(frac=1, random_state=random_state).reset_index(drop=True)
    num_bins = int(np.floor(1 + np.log2(len(data))))  # bins数量(Sturges规则)
    data.loc[:, "bins"] = pd.cut(data["target"], bins=num_bins, labels=False)  # 根据'target'列分箱
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, "fold"] = f

    data = data.drop("bins", axis=1)
    return data

In [3]:
data_train = pd.read_csv('../datasets/train.csv')
data_train.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [4]:
df_train_oof = create_folds(data_train, 5)
df_train_oof.head()

,id,url_legal,license,excerpt,target,standard_error,fold
0,0cdf53452,NaN,NaN,It so happened that Worcester was captain of B...,-2.462000,0.521126,1
1,e7010d02a,NaN,NaN,Some weeks passed by before Nellie was allowed...,0.507237,0.523720,3
2,56e590582,https://kids.frontiersin.org/article/10.3389/f...,CC BY 4.0,"Sometimes, bacteria swim or float freely in li...",-0.573611,0.463228,1
3,c9af1974b,https://simple.wikipedia.org/wiki/Air_navigation,CC BY-SA 3.0 and GFDL,Air navigation is navigation while flying. It ...,-0.032950,0.500576,1
4,29cd28197,https://en.wikipedia.org/wiki/Light-year,CC BY-SA 3.0,The light-year is a unit of length used to exp...,-3.256312,0.581264,0


In [5]:
fold_list = [0, 1, 2, 3, 4]

for fold in fold_list:
    train_df = df_train_oof[df_train_oof['fold'] != fold]  # 训练数据集
    val_df = df_train_oof[df_train_oof['fold'] == fold]  # 验证数据集
    print(train_df.shape)
    print(val_df.shape)

(2267, 7)
(567, 7)
(2267, 7)
(567, 7)
(2267, 7)
(567, 7)
(2267, 7)
(567, 7)
(2268, 7)
(566, 7)
